# Kapitel 4: Feature Engineering

## 🎯 Ziel
Das Ziel des Feature Engineerings war, alle relevanten Informationen zu extrahieren, die für die Vorhersage des Anlasses (z. B. „Date Night“, „Ladys Night“) nützlich sind – ohne unnötiges Rauschen.

## Encoding-Fokus & Beschreibung
### Numerische Features:

- duration_scaled: Dauer des Films (in Minuten), skaliert per MinMaxScaler, da die Werte stark variieren (z. B. Serien vs. Kurzfilme).
- agerating_scaled: Altersfreigabe (z. B. PG-13 → 13), ebenfalls skaliert. Wichtig für Events wie „Family Night“ oder „Kids Night“.
### Textuelle Features (via TF-IDF):

- title und description wurden getrennt TF-IDF-vektorisiert, um semantische Begriffe wie „love“, „horror“, „christmas“ etc. zu erfassen.
- Beispiel: Beschreibung mit „A romantic story set in winter“ → hoher Score für „romantic“ und „winter“, was z. B. „Date Night“ oder „Christmas“ begünstigt.
### Genre-Features:

Zwar encoded (One-Hot mit MultiLabelBinarizer), aber bewusst nicht für das finale Modell verwendet, da diese Information bereits stark in der Beschreibung reflektiert wird und zu Overfitting führen kann.

## Feature Engineering für Zielvariable
- Zielvariable: anlass_label – das erste Element aus der anlass_rule-Liste.
- Die Label wurden vorher regelbasiert und mit TF-IDF-Keywords ergänzt – dadurch hohe Qualität und Konsistenz.
- Wichtig für den Business-Case: Wer den Anlass kennt, kann zielgerichtete Empfehlungen geben – z. B. „Ladys Night“ → romantische Komödien mit weiblicher Hauptrolle.

## Feature Importance
Die Feature Importance wurde aus dem RandomForestClassifier extrahiert und aggregiert auf:

- Beschreibung (TF-IDF): klar dominant
- Titel: moderat wichtig
- Dauer & Altersfreigabe: geringer, aber signifikant
-> Das zeigt: Der Text (vor allem Beschreibung) ist der zentrale Prädiktor, was für Content-getriebene Empfehlungssysteme absolut plausibel ist.

# Kapitel 5: Modellierung

- Modellwahl: RandomForestClassifier
    - Modell: Random Forest Classifier

- Warum Random Forest?
    - Robust gegenüber Overfitting
    - Kann mit vielen Features umgehen (z. B. Genres, Plattformen)
    - Gibt Feature Importance zurück (nützlich für Business Insights)
    - Funktioniert gut bei Multi-Class Classification – wie hier mit verschiedenen Anlässen


## Grund für Wahl:
- funktioniert gut mit hoher Dimensionalität (TF-IDF),
- robust gegen Ausreißer & Skalenunterschiede,
- liefert direkt Feature Importances (wichtig für Analyse),
- keine starke Annahmen über die Datenverteilung nötig.

## Bewertungsmetriken
Genutzt wurde:
- Accuracy: Gesamt-Trefferquote
- Precision, Recall, F1-Score pro Klasse
- Macro Average: wichtig bei ungleich verteilten Klassen (jede Klasse zählt gleich)
→ Warum kein ROC-AUC?
Da es sich um ein Multiclass-Problem mit mehr als 2 Klassen handelt, ist F1-Score + Macro Average deutlich interpretierbarer und nützlicher.

Die Funktion train_anlass_model(...) trainiert nur das Modell, sie berechnet (noch) keine fehlenden Anlässe.

Was sie genau macht:
📦 Feature-Auswahl: Wählt numerische Spalten aus (genre_*, platform_*, duration_scaled, agerating_scaled)
🧹 Trainingsdaten filtern: Nur Zeilen, bei denen anlass_rule bereits vergeben ist (len(x) > 0)
🎯 Zielspalte erzeugen: Das erste Element der anlass_rule-Liste wird zur Target-Variable anlass_label
🧠 Modell trainieren: Ein RandomForestClassifier lernt auf diesen Daten, welcher Anlass zu einem Film passt
📈 Modell evaluieren: Ausgabe eines classification_report, also Performance des Modells auf Testdaten
🔁 Modell & Feature-Liste zurückgeben: Für spätere Anwendung

# kapitel 6: Modeltraining & Hyperparameter

## Split
- train_test_split mit 80 % Training, 20 % Test
- zufällig, aber durch random_state=42 reproduzierbar
- wichtig: anlass_label wurde vorher extrahiert, nur Filme mit gültigem Label wurden für das Training verwendet

## Hyperparameter
Genutzt:
- n_estimators=100 (Standard)
- class_weight='balanced': automatisch ausgleichend bei ungleich verteilten Klassen
- random_state=42: für Reproduzierbarkeit
Noch kein GridSearch oder Optuna-Tuning → kann als nächster Schritt erfolgen



# Kapitel 7: Evaluation

Die Bewertung des Modells erfolgte mit Fokus auf den **Macro F1-Score**, da das Datenset eine **unausgeglichene Klassenverteilung** aufweist. In solchen Fällen ist Accuracy als Metrik ungeeignet, da sie durch häufig vorkommende Klassen verzerrt werden kann. Auch Precision und Recall alleine liefern keine robuste Aussage über die Gesamtleistung.

Der **Macro F1-Score** berechnet den F1-Wert **gleichgewichtet über alle Klassen hinweg**, unabhängig davon, wie oft eine Klasse im Datenset vorkommt. Dadurch wird sichergestellt, dass das Modell **alle Anlasse gleichermaßen** gut erkennen soll – nicht nur die häufigsten.

Die Ergebnisse des trainierten Modells lauten wie folgt:

- **Macro F1-Score:** `0.863`
- **Weitere Klassenmetriken** (optional):
  - Family Night: F1 = 0.93
  - Educational Night: F1 = 0.89
  - Date Night: F1 = 0.88
  - Kids Night: F1 = 0.80
  - Halloween Night: F1 = 0.78
  - Christmas: F1 = 0.83

Diese Werte zeigen, dass das Modell in der Lage ist, **auch seltener vorkommende Anlässe** zuverlässig vorherzusagen.

**Fazit:** Die Wahl des Macro-F1-Scores als Zielmetrik ist in diesem Anwendungsfall gerechtfertigt und liefert eine faire Bewertung des Modells.


dash_bootstrap_components  muss installiert werden wenn nicht vorhanden